# Import Modules

In [33]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import networkx as nx

In [34]:
# Import the data
df = pd.read_csv('../RVF_ATX_PID_HZ-2020-07.tsv', sep='\t')
# filter out certain columns
df = df.drop(columns=['local_date', 'local_hour', 'gender', 'age', 'full_panel_reweighted_sag_score', 'home_zip'])
df.head()

,persistentid,venueid,utc_date,utc_hour,dwell
0,5903d26cdcecbd13590c8fe594de785f19b16004e19156...,40b52f80f964a52051001fe3,2020-07-18,22,60
1,cc371dcb888f9ec0ab9a4ecabc4d49e51288313fb17a47...,40b52f80f964a52051001fe3,2020-07-18,22,1011
2,21e09ed692d56697e3c26b777a53a411cd21bed5527c5b...,40b52f80f964a52051001fe3,2020-07-25,0,1058
3,e170f93db0ea4930ea2c0d2167feddb1b4fe2b5412d826...,40b52f80f964a52051001fe3,2020-07-25,18,30
4,63164c43b459b4260a338979948ead113855ddaafb9d52...,40b52f80f964a52051001fe3,2020-07-29,18,763


In [35]:
# filter out rows that are not within a range

# Define the date range
start_date = pd.to_datetime('2020-07-01')
end_date = pd.to_datetime('2020-07-01')
dwell_time_threshold = 60 # in minutes

# Filter rows within the date range
filtered_df = df[(pd.to_datetime(df['utc_date']) >= start_date) & (pd.to_datetime(df['utc_date']) <= end_date) & (df['dwell'] >= dwell_time_threshold) & (df['utc_hour'] <= 24)]
# add a new column to dataframe that represents minutes
filtered_df['formatted_utc_hour'] = filtered_df['utc_hour'].astype(int)
filtered_df['arrival_time'] = pd.to_datetime(filtered_df['utc_date']) + pd.to_timedelta(filtered_df['formatted_utc_hour'], unit='h')

filtered_df['departure_time'] = filtered_df['arrival_time'] + pd.to_timedelta(filtered_df['dwell'], unit='m')
filtered_df = filtered_df.sort_values(by=['arrival_time'])
filtered_df = filtered_df.reset_index(drop=True)
print(filtered_df.shape)
filtered_df.head()

(19085, 8)


/tmp/ipykernel_27313/1807461848.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_df['formatted_utc_hour'] = filtered_df['utc_hour'].astype(int)
/tmp/ipykernel_27313/1807461848.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_df['arrival_time'] = pd.to_datetime(filtered_df['utc_date']) + pd.to_timedelta(filtered_df['formatted_utc_hour'], unit='h')
/tmp/ipykernel_27313/1807461848.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try us

,persistentid,venueid,utc_date,utc_hour,dwell,formatted_utc_hour,arrival_time,departure_time
0,abd6f1a8241d1bfce62ba7f0407d2f214658c28481123e...,509d2e25f31cea82a7eda85e,2020-07-01,0,540,0,2020-07-01,2020-07-01 09:00:00
1,4d0fbf4b0088fbaa6ff0e9dbc3b766a1889c4b471cca11...,4b43f7adf964a52049ef25e3,2020-07-01,0,1521,0,2020-07-01,2020-07-02 01:21:00
2,9930410ed7a7bc6b7dd457c065559d72bcf7df18395cd8...,4b02cb6af964a520f34922e3,2020-07-01,0,309,0,2020-07-01,2020-07-01 05:09:00
3,e4c80e0ee166ee00313709edb6d78944a30951bd9c6fba...,4d239e3853e4a093d6ae8077,2020-07-01,0,182,0,2020-07-01,2020-07-01 03:02:00
4,34d414ab3f9de469b0b0e33cbc7014d257298737b721d4...,592eaa308496ca3ad6dcf221,2020-07-01,0,829,0,2020-07-01,2020-07-01 13:49:00


In [36]:
# iterate through df and store values in a list to be easily accessible
# create a hashmap where node is the key and value if information (list of (location, timespan) tuples)
node_information = {}
for row in filtered_df.itertuples():
    persistentid = 1
    venueId = 2
    arrival_time = 7
    departure_time = 8
    device = row[persistentid]
    venue = row[venueId]
    time_of_arrival = row[arrival_time]
    if device in node_information:
        node_information[device].append((venue, time_of_arrival))
    else:
        node_information[device] = [(venue, time_of_arrival)]
for node in node_information:
    # sort by time
    node_information[node].sort(key=lambda x: x[1])
print(node_information)

{'abd6f1a8241d1bfce62ba7f0407d2f214658c28481123ec9b7785386463bc60c': [('509d2e25f31cea82a7eda85e', Timestamp('2020-07-01 00:00:00')), ('508985a4067de3ac493b2d0f', Timestamp('2020-07-01 00:00:00')), ('4b23de18f964a5209d5b24e3', Timestamp('2020-07-01 00:00:00')), ('4c9d0d8d74109eb014e41415', Timestamp('2020-07-01 01:00:00')), ('4b06e95bf964a520cdf222e3', Timestamp('2020-07-01 01:00:00')), ('49c2e908f964a52040561fe3', Timestamp('2020-07-01 18:00:00')), ('4aa7cac8f964a5203a4d20e3', Timestamp('2020-07-01 18:00:00')), ('4bb0d64af964a520a5613ce3', Timestamp('2020-07-01 18:00:00')), ('4a870ba3f964a5204e0220e3', Timestamp('2020-07-01 19:00:00')), ('4a57c7ccf964a520c9b61fe3', Timestamp('2020-07-01 19:00:00')), ('443537b6f964a52006321fe3', Timestamp('2020-07-01 19:00:00')), ('519f9544498e6232e91ce8b6', Timestamp('2020-07-01 23:00:00')), ('4be624eecf200f47c14c143c', Timestamp('2020-07-01 23:00:00'))], '4d0fbf4b0088fbaa6ff0e9dbc3b766a1889c4b471cca11f667de4526a2f55510': [('4b43f7adf964a52049ef25e3',

In [37]:
# get all persistentids where time of arrival is at hour 0
starting_date_time = pd.to_datetime('2020-07-01 00:00:00')
potential_starting_nodes = []
for node in node_information:
    if node_information[node][0][1] == starting_date_time:
        potential_starting_nodes.append(node)
print(f"{len(potential_starting_nodes)}: {potential_starting_nodes}")
# randomly sample 10% of nodes to start covid with 
starting_nodes = np.random.choice(potential_starting_nodes, int(len(potential_starting_nodes) * 0.2), replace=False)
print(starting_nodes)


949: ['...']


In [38]:
# add all devices as nodes to Graph
# get all devices
G = nx.Graph()
for device in starting_nodes:
    # add device as a node to G
    G.add_node(device)
nx.write_gexf(G, "t0.gexf")
    

In [39]:
# iterate over all devices starting at hour 1 and to hour 6
# if 2 people meet, add an edge between them
infected = set(starting_nodes)
hour = 1
while(hour <= 24):
    for node in node_information:
        if node in infected:
            continue
        # get all places node has been to for that hour
        places = []
        for place in node_information[node]:
            if place[1].hour == hour:
                places.append(place[0])
        
        # get all places that infected people have been to for that hour
        # if there is overlap, then create an edge
        new_infected = set()
        for infected_node in infected:
            infected_places = []
            for place in node_information[infected_node]:
                if place[1].hour == hour:
                    infected_places.append(place[0])
            if set(places).intersection(infected_places):
                G.add_edge(node, infected_node)
                new_infected.add(node)
        # copy new_infected to infected
        infected.update(new_infected)
    # save the graph
    nx.write_gexf(G, f"t{hour}.gexf")
    hour += 1